In [1]:
from buildingmotif import BuildingMOTIF
bm = BuildingMOTIF("sqlite://") 

In [46]:
from rdflib import Namespace, Literal
from buildingmotif.dataclasses import Model

# create the namespace
BLDG = Namespace('test:building/')

# Create the model! This will raise an exception if the namespace is not syntactically valid.
model = Model.create(BLDG, description="This is a test model for a simple building") 

In [3]:
print(model.graph.serialize())

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<test:building/> a owl:Ontology .




# Loading Libraries

In [4]:
# load a library
from buildingmotif.dataclasses import Library
brick = Library.load(ontology_graph="Brick-full.ttl")

# print the first 10 templates
print("The Brick library contains the following templates:")
for template in brick.get_templates()[:10]:
    print(f"  - {template.name}")
    

/Users/lazlopaul/opt/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(
2023-01-13 18:37:52,212 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2023-01-13 18:37:52,212 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject


The Brick library contains the following templates:
  - https://brickschema.org/schema/Brick#Displacement_Flow_Air_Diffuser
  - https://brickschema.org/schema/Brick#Max_Unoccupied_Heating_Supply_Air_Flow_Setpoint_Limit
  - https://brickschema.org/schema/Brick#Discharge_Air_Duct_Pressure_Status
  - https://brickschema.org/schema/Brick#Pump_Status
  - https://brickschema.org/schema/Brick#CO2_Alarm
  - https://brickschema.org/schema/Brick#Dimmer
  - https://brickschema.org/schema/Brick#IDF
  - https://brickschema.org/schema/Brick#Max_Occupied_Heating_Discharge_Air_Flow_Setpoint_Limit
  - https://brickschema.org/schema/Brick#Condenser_Water_Pump
  - https://brickschema.org/schema/Brick#Min_Discharge_Air_Static_Pressure_Setpoint_Limit


# Exploring a Template

In [5]:
# import this to make writing URIs easier
from buildingmotif.namespaces import BRICK


"""
# get template
ahu_template = brick.get_template_by_name(BRICK.AHU)



print(ahu_template)
# print template parameters

print("The template has the following parameters:")
for param in ahu_template.parameters:
    print(f"  {param}")
"""

'\n# get template\nahu_template = brick.get_template_by_name(BRICK.AHU)\n\n\n\nprint(ahu_template)\n# print template parameters\n\nprint("The template has the following parameters:")\nfor param in ahu_template.parameters:\n    print(f"  {param}")\n'

In [6]:
#print(ahu_template.body.serialize())


# Evaluating a Template

In [7]:
"""
ahu_name = "AHU_AC"
ahu_binding_dict = {"name": BLDG[ahu_name]}
ahu_graph = ahu_template.evaluate(ahu_binding_dict)

# ahu_graph is just an instance of rdflib.Graph
print(ahu_graph.serialize())
"""

'\nahu_name = "AHU_AC"\nahu_binding_dict = {"name": BLDG[ahu_name]}\nahu_graph = ahu_template.evaluate(ahu_binding_dict)\n\n# ahu_graph is just an instance of rdflib.Graph\nprint(ahu_graph.serialize())\n'

# Adding Evaluated Templates to the Model

In [8]:
"""
model.add_graph(ahu_graph)
print(model.graph.serialize())
"""

'\nmodel.add_graph(ahu_graph)\nprint(model.graph.serialize())\n'

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [113]:
df=pd.read_csv('entity_relations_v4.csv')
filename  = '/Users/lazlopaul/Desktop/VolttronSemantics/files/entity_table.csv'
topics = pd.read_csv(filename)[['URI','base_topic']]
df = df.merge(topics, left_on = 'subjectURI', right_on = 'URI')

In [114]:
df['pointURI'] = np.zeros(len(df.index))
df['topic'] = np.zeros(len(df.index))

In [115]:
for i in df.index:
    if df.loc[i, 'relation'] == 'hasPoint':
        df.loc[i, 'pointURI'] = df.loc[i, 'subjectURI'] + '.' + df.loc[i, 'objectURI']
        df.loc[i, 'topic'] = df.loc[i, 'base_topic'] + '/' + df.loc[i, 'objectURI']

df.head()

,subjectURI,brick_class,relation,objectURI,brick_class.1,URI,base_topic,pointURI,topic
0,FCU1,FCU,feeds,Zone1,HVAC_Zone,FCU1,sensibo/FCU1,0.0,0.0
1,FCU1,FCU,hasPoint,targetTemperature,Air_Temperature_Setpoint,FCU1,sensibo/FCU1,FCU1.targetTemperature,sensibo/FCU1/targetTemperature
2,FCU1,FCU,hasPoint,timeOfTemperature,Time_Parameter,FCU1,sensibo/FCU1,FCU1.timeOfTemperature,sensibo/FCU1/timeOfTemperature
3,FCU1,FCU,hasPoint,temperature,Air_Temperature_Sensor,FCU1,sensibo/FCU1,FCU1.temperature,sensibo/FCU1/temperature
4,FCU1,FCU,hasPoint,on,On_Off_Status,FCU1,sensibo/FCU1,FCU1.on,sensibo/FCU1/on


In [108]:
import re
def uni_encode(mystring):
    mystring=str(mystring)
    mystring=re.sub(r'[^\w\s]', '', mystring)
    mynewstring=mystring.replace(" ", "_")
    return mynewstring
df=df.applymap(lambda x: uni_encode(x) if pd.notnull(x) else x)

In [109]:

#print(equipment_template)

In [110]:
length_dataframe=len(df)
#print(length_dataframe)

for row_df in range(0,length_dataframe):
    for column_df in [0,3]:
        brick_class=df.iloc[row_df,column_df+1]
        dummy_code = f"equipment_template= brick.get_template_by_name(BRICK.{brick_class})"
        exec(dummy_code)
        equipment_name =df.iloc[row_df,column_df]
        equipment_binding_dict = {"name": BLDG[equipment_name]}
        equipment_graph = equipment_template.evaluate(equipment_binding_dict)
        model.add_graph(equipment_graph)




In [111]:
for row_df in range(0,length_dataframe):
    equipment_name1 =df.iloc[row_df,0]
    equipment_name2 =df.iloc[row_df,3]
    BRICK_relation=df.iloc[row_df,2]
    dummy_code=f"model.graph.add((BLDG['{equipment_name1}'], BRICK.{BRICK_relation}, BLDG['{equipment_name2}']))"
    exec(dummy_code)
        
        

In [15]:
"""

# templates
oa_ra_damper_template = brick.get_template_by_name(BRICK.Damper)
fan_template = brick.get_template_by_name(BRICK.Supply_Fan)
clg_coil_template = brick.get_template_by_name(BRICK.Cooling_Coil)

# add fan
fan_name = f"{ahu_name}-Fan"
fan_binding_dict = {"name": BLDG[fan_name]}
fan_graph = fan_template.evaluate(fan_binding_dict)
model.add_graph(fan_graph)

# add outdoor air/return air damper
oa_ra_damper_name = f"{ahu_name}-Damper"
oa_ra_damper_binding_dict = {"name": BLDG[oa_ra_damper_name]}
oa_ra_damper_graph = oa_ra_damper_template.evaluate(oa_ra_damper_binding_dict)
model.add_graph(oa_ra_damper_graph)

# add clg coil
clg_coil_name = f"{ahu_name}-Clg_Coil"
clg_coil_binding_dict = {"name": BLDG[clg_coil_name]}
clg_coil_graph = clg_coil_template.evaluate(clg_coil_binding_dict)
model.add_graph(clg_coil_graph)

# connect fan, damper, and clg coil to AHU
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[oa_ra_damper_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[fan_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[clg_coil_name]))
"""
# print model to confirm components were added and connected
print(model.graph.serialize())


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<test:building/> a owl:Ontology .

<test:building/Bldg> a brick:Building ;
    brick:hasPart <test:building/AdminSpace>,
        <test:building/Office> .

<test:building/FCU1> a brick:FCU ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPoint <test:building/fanLevel>,
        <test:building/mode>,
        <test:building/on>,
        <test:building/targetTemperature>,
        <test:building/temperature>,
        <test:building/timeOfTemperature> .

<test:building/FCU2> a brick:FCU ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPoint <test:building/fanLevel>,
        <test:building/mode>,
        <test:building/on>,
        <test:building/targetTemperature>,
        <test:building/temperature>,
        <test:building/timeOfTemperature> .

<test:building/FCU3> a brick:FCU ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPoint <test:building/fanLevel>,
        <test

In [16]:
#save model
model.graph.serialize(destination="sample_building_model.ttl")

<Graph identifier=4317100e-9fbf-4356-821a-ce11d56b6993 (<class 'rdflib.graph.Graph'>)>

# LP: Adding volttron topics using ref schema

In [131]:
from buildingmotif.namespaces import REF
model.graph.bind("ref", REF)

In [22]:
# VLTT = Namespace('https://www.doesntexist.org/volttronsemantics#')
# model.graph.bind("VLTT", VLTT)
# vltt = Library.load(ontology_graph="../../files/volttron_templates.ttl")
# vltt_ref_template = vltt.get_template_by_name(VLTT.VolttronReference)

In [122]:
ref_template = brick.get_template_by_name(REF.TimeseriesReference)

In [132]:
ref_name = f"volttrontopicname"
ref_binding_dict = {"name": BLDG[ref_name], "p12":  Literal('this/is/topic')}
ref_graph = ref_template.evaluate(ref_binding_dict)
ref_graph.serialize()

'@prefix ref: <https://brickschema.org/schema/Brick/ref#> .\n\n<test:building/volttrontopicname> a ref:TimeseriesReference ;\n    ref:hasTimeseriesId "this/is/topic" .\n\n'

In [137]:
for i, row in df.iterrows():
    if row['relation'] == 'hasPoint':
        ref_binding_dict = {"name": BLDG[row['pointURI'] + '.id'], "p12":  Literal(row['topic'])}
        model.add_graph(ref_template.evaluate(ref_binding_dict))
        model.graph.add((BLDG[row['pointURI']], REF['hasTimeseriesReference'], BLDG[row['pointURI'] + '.id']))

In [138]:
print(model.graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <https://brickschema.org/schema/Brick/ref#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .

<test:building/> a owl:Ontology .

<test:building/Bldg> a brick:Building ;
    brick:hasPart <test:building/AdminSpace>,
        <test:building/Office> .

<test:building/Boiler2> a brick:Boiler ;
    brick:feeds <test:building/HeatEx2> .

<test:building/FCU1> a brick:FCU ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPoint <test:building/fanLevel>,
        <test:building/mode>,
        <test:building/on>,
        <test:building/targetTemperature>,
        <test:building/temperature>,
        <test:building/timeOfTemperature> .

<test:building/FCU1.fanLevel> ref:hasTimeseriesReference <test:building/FCU1.fanLevel.id> .

<test:building/FCU1.mode> ref:hasTimeseriesReference <test:building/FCU1.mode.id> .

<test:building/FCU1.on> ref:hasTimeseriesReference <test